# **Example 4** - Measures and Strategies

In this notebook we will show you all the different options and inputs for creating different measures and how to create a strategy from the different measures. 

**Measures** in FloodAdapt can be categorized into **two** different categories, which we will cover all in this notebook:

**1. Hazard** - Mitigate the hazard directly

- Floodwall
- Pump
- Green Infrastructure

**2. Impact**  - Mitiage the impacts of the hazard on the assets

- Elevate properties 
- Buyout Properties
- Floodproof Properties

**Strategies** consist of multiple measures combined for example: ***StrategyXYZ* = *Floodwall + Elevated Homes + Pumps***

If you want to learn in more detail about measures and what a strategy is please read the sections on [**Measures**](https://deltares-research.github.io/FloodAdapt/4_user_guide/measures/) and [**Strategies**](https://deltares-research.github.io/FloodAdapt/4_user_guide/strategy/)  in the FloodAdapt documentation.

## **Step 0**: Import packages

In [ ]:
from pathlib import Path

from flood_adapt.objects.measures.measures import *
from flood_adapt import unit_system as us

## **Step 1:** Create Hazard Measures
We can divide the hazard measures into: 
1. **Hydraulic** (grey) measures -  structures such as levees, pumps, seawalls that protect infrastructure and residents
2. **Green infrastructure** measures - integrate natural elements like parks, green roofs, and permeable pavements into city

To create a measure in Floodadapt we need to create a **Measure Object** with the FloodAdapt classes. Each object has the following parameters:

**Basic** inputs for <u>all</u> types of hazard measures are:
- ***name***: The name of the measure
- ***description*** (optionl): A description of the measure
- ***type***: The type of measure
- ***selection type***: The type of spatial shape of the measure
- ***polygon file***: The file path to the spatial file of the measure

**Floodwall** specific inputs are:
- ***elevation***:

**Pump** specific inputs are:
- ***discharge***: 

**Greening** specific inputs are:
- ***volume***: 
- ***height*** (optional): 
- ***aggregation_area_type*** (optional): 
- ***aggregation_area_name*** (optional): 
- ***percent_area*** (optional):

Below we will create an object for each hazard measure.

In [ ]:
# Create a measure object for a Floodwall
attrs_floodwall = FloodWall(
        name= "Seawall_12ft",
        description = "12ft Seawall",
        type = MeasureType.floodwall,
        selection_type = SelectionType.polyline,
        polygon_file = str(r"C:\Users\rautenba\repos\Database\charleston_full\input\measures\seawall\seawall.geojson"),
        elevation = us.UnitfulLength(value=12, units=us.UnitTypesLength.feet))

# Create a measure object for a Pump
attrs_pump = Pump(
        name= "Pump",
        description = "Pump",
        type = MeasureType.pump,
        selection_type = SelectionType.polygon,
        polygon_file = str(r""),
        discharge = us.UnitfulDischarge(value=1, units=us.UnitTypesDischarge.cfs))

# Create a measure object for Green infrastructure
attrs_green_infra = GreenInfrastructure(
        name= "green_infrastructure_storage",
        description = "Storage through green infrastructure",
        type = MeasureType.greening,
        selection_type = SelectionType.polygon,
        polygon_file = str(r"")
        volume = us.UnitfulVolume(value = 3, units = us.UnitTypesVolume.cf),
        height = us.UnitfulHeight(value = 3, units = us.UnitTypesLength.feet),
        aggregation_area_type: "", #TODO
        aggregation_area_name: "", #TODO
        percent_area: Optional[float] = #TODO
        )

## **Step 2:** Create Impact Measures
To create a measure in Floodadapt we need to create a **Measure Object** with the FloodAdapt classes. Each object has the following parameters:

**Basic** inputs for <u>all</u> types of impact measures are:
- ***name***: The name of the measure
- ***description*** (optionl): A description of the measure
- ***type***: The type of measure
- ***selection type***: The type of spatial shape of the measure
- ***polygon file***: The file path to the spatial file of the measure
- ***property_type***: The building type, e.g. residential, commercial, depends on you Delft-FIAT model classification
- ***aggregation_area_type*** (optional):
- ***aggregation_area_name*** (optional):

**Elevate** specific inputs are:
- ***elevation***:

**Buy out** specific inputs are:
- ***None***

**Floodproof** specific inputs are:
- ***elevation***:

Below we will create an object for each impact measure.

In [ ]:
# Create a measure object for Elevating Buildings
attrs_floodwall = Elevate(
        name= "elevate_homes_2ft",
        description = "Elevate all residential buildings in aggregation area 1 by 2ft.",
        type = MeasureType.elevate_properties,
        selection_type = SelectionType.aggregation_area,
        property_type = "residential",
        polygon_file =str(r"C:\Users\rautenba\repos\Database\charleston_full\input\measures\raise_property_polygon\raise_property_polygon.geojson"),
        elevation = us.UnitfulLength(value=2, units=us.UnitTypesLength.feet))

# Create a measure object for Buying out Buildings
attrs_pump = Buyout(
        name= "buyout_all_buildings",
        description = "Buyout all buildings in a specific area.",
        type = MeasureType.pump,
        selection_type = SelectionType.polygon,
        property_type = "ALL",
        polygon_file = str(r""),

# Create a measure object for Floodproofing Buildings
attrs_green_infra = FloodProof(
        name= "floodproof_all_com",
        description = "Floodproofing all commercial buildings.",
        type = MeasureType.greening,
        selection_type = SelectionType.all,
        polygon_file = str(r""),
        property_type = "commercial",
        elevation = us.UnitfulLength(value=2, units=us.UnitTypesLength.feet))
        )

#### Selection Types and property type
describe all the options


### **2. Green Infrastructure Measures**

### **3. Building Level Measures**